# Required Modules

In [ ]:
pip install folium

In [ ]:
pip install ray

In [ ]:
pip install geopy

In [ ]:
pip install warning

In [35]:
import os
import folium
from branca.element import Figure
import pandas as pd
import GeneticAlgoWithModes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Inter City Transportation Cost via Different Modes

In [36]:
data = pd.read_excel("TransportData.xlsx",sheet_name='RoadFare')
data.reset_index(inplace=True)
data1 = data.iloc[:,2:]
data1.index = data1.columns.values
data1

,Bangalore,Jaipur,Varanasi,Udaipur,Delhi,Mumbai
Bangalore,0,2026,2001,1724,2166,980
Jaipur,2026,0,849,393,287,1146
Varanasi,2001,849,0,1154,834,1524
Udaipur,1724,393,1154,0,661,2000
Delhi,2166,287,834,661,0,1415
Mumbai,980,1146,1524,2000,1415,0


In [37]:
data_Air = pd.read_excel("AirFare.xlsx",sheet_name='AirFare')
data_Air.reset_index(inplace=True)
data_Air1 = data_Air.iloc[:,2:]
data_Air1.index = data_Air1.columns.values
data_Air1

,Bangalore,Jaipur,Varanasi,Udaipur,Delhi,Mumbai
Bangalore,0,1990,2000,9777,7068,4758
Jaipur,1990,0,8014,4114,3331,5111
Varanasi,2000,8014,0,11576,5018,6067
Udaipur,9777,4114,11576,0,3849,1200
Delhi,7068,3331,5018,3849,0,5535
Mumbai,4758,5111,6067,1200,5535,0


In [38]:
data_Train = pd.read_excel("AirFare.xlsx",sheet_name='TrainFare')
data_Train.reset_index(inplace=True)
data_Train1 = data_Train.iloc[:,2:]
data_Train1.index = data_Train1.columns.values
data_Train1

,Bangalore,Jaipur,Varanasi,Udaipur,Delhi,Mumbai
Bangalore,0,3200,2995,3600,5425,790
Jaipur,3200,0,1685,1015,460,2070
Varanasi,2995,1685,0,1940,1675,2365
Udaipur,3600,1015,1940,0,1490,1850
Delhi,5425,460,1675,1490,0,4110
Mumbai,790,2070,2365,1850,4110,0


# Find route to cover all the cities from withstarting and ending poitn as Bnaglore

In [39]:
for i in [10,60]:
    ga_obj= GeneticAlgoLibrary1.OverallGaRun(noverall=10,
                                       number_of_cities=6,
                                       initial_pop_size=10000,
                                       nelite=10,
                                       percentage_to_crossover=20,
                                       percentage_to_mutate=20,
                                       dist_mat=data1,
                                       air_cost_mat=data_Air1,
                                       train_cost_mat=data_Train1
                                              )
    best_sol = ga_obj.run_overall_ga()

Cities Mapping: 100%|██████████| 6/6 [00:00<00:00, 9664.29it/s]


Starting 10 iterations of the GA
(fitness_function pid=20917) 
(fitness_function pid=20917) 
(fitness_function pid=20916) 
(fitness_function pid=20914) 


Cities Mapping: 100%|██████████| 6/6 [00:00<00:00, 55067.45it/s]

-----------------------------------------------------------------
Best solution for initial population size of 10000 and number 
            of runs 10 is 
                 Route                                   Cost
0  [2, 4, 1, 3, 5, 0]  (5504, ,air,road,road,road,air,train)
Generated 20(16.667%) of the 120 solutions
Total Cost to cover the final route of 
 Bangalore => Varanasi => Delhi => Jaipur => Udaipur => Mumbai => Bangalore is Rs (5504, ',air,road,road,road,air,train').
             (Generated from initial population size of 10000)
-----------------------------------------------------------------
Starting 10 iterations of the GA


(fitness_function pid=20914) 
(fitness_function pid=20917) 
(fitness_function pid=20916) 


Crossover: 100%|██████████| 3/3 [00:00<00:00, 768.00it/s]


-----------------------------------------------------------------
Best solution for initial population size of 10000 and number 
            of runs 10 is 
                 Route                                   Cost
0  [2, 4, 1, 3, 5, 0]  (5504, ,air,road,road,road,air,train)
Generated 20(16.667%) of the 120 solutions
Total Cost to cover the final route of 
 Bangalore => Varanasi => Delhi => Jaipur => Udaipur => Mumbai => Bangalore is Rs (5504, ',air,road,road,road,air,train').
             (Generated from initial population size of 10000)
-----------------------------------------------------------------


# Plotting optimized route with mode of transport on map

**GA major loop**

In [56]:
final_route = [0]
final_route.extend(best_sol[0]["Route"].values[0])
# final_route.append(0)

# creating a lisot with naplot_objectes
final_route_naplot_objectes = []
for i in final_route:
    final_route_naplot_objectes.append(data1.index[i])

lat_lon = pd.read_csv("city_lat_lon.csv")


fig=Figure(height=1000,width=750)
plot_object=folium.Map(location=[12.9716, 77.5946],tiles='cartodbpositron',zoom_start=5)
fig.add_child(plot_object)
modes=best_sol[1][1].split(',')
modes.pop(0)
modes.insert(0,0)

print("Optimized route: ")

for i,a in enumerate(final_route_naplot_objectes):
    if(i<len(final_route_naplot_objectes)-1):
        print(a + ' to ' + final_route_naplot_objectes[i+1] +'--------->'+ modes[i+1])
        
print("Total cost: Rs ", best_sol[1][0])

coordinates = []
for a,i in enumerate(final_route_naplot_objectes):
    lat,lon = lat_lon[lat_lon.City == i]["Lat"].values[0],lat_lon[lat_lon.City == i]["Lon"].values[0]
    folio_group_object = folium.FeatureGroup("Route")
    coordinates.append([lat,lon])
    newLine=coordinates[a-1],coordinates[a]
    if(modes[a]=='road'):
        line_1=folium.vector_layers.PolyLine(newLine,popup='<b>Best path</b>',tooltip='Road',color='red',
                                         weight=2).add_to(folio_group_object)
        line_1.add_to(plot_object)
    elif(modes[a]=='train'):
        line_1=folium.vector_layers.PolyLine(newLine,popup='<b>Best path</b>',tooltip='Train',color='green',
                                         weight=2).add_to(folio_group_object)
        line_1.add_to(plot_object)
    elif(modes[a]=='air'):
        line_1=folium.vector_layers.PolyLine(newLine,popup='<b>Best path</b>',tooltip='Air',color='blue',
                                         weight=2).add_to(folio_group_object)
        line_1.add_to(plot_object)
        
plot_object

Optimized route: 
Bangalore to Varanasi--------->air
Varanasi to Delhi--------->road
Delhi to Jaipur--------->road
Jaipur to Udaipur--------->road
Udaipur to Mumbai--------->air
Mumbai to Bangalore--------->train
Total cost: Rs  5504
